In [3]:
!pip install geopandas
!pip install cython
!pip install git+https://github.com/jcrudy/choldate.git
!pip install knockpy

  Cloning https://github.com/jcrudy/choldate.git to /tmp/pip-req-build-nfm3hhgh
  Running command git clone --filter=blob:none --quiet https://github.com/jcrudy/choldate.git /tmp/pip-req-build-nfm3hhgh
  Resolved https://github.com/jcrudy/choldate.git to commit 0d92a523f8da083031faa0eb187a7b0f287afe69
  Preparing metadata (setup.py) ... done
  Created wheel for choldate: filename=choldate-0.1.0-cp310-cp310-linux_x86_64.whl size=98539 sha256=6735c6a06dfd31515ba9579ca2f297edacb1823902a399b248aeef3bff1e38ca
  Stored in directory: /tmp/pip-ephem-wheel-cache-fkvp0brw/wheels/25/4a/48/b3236da795d0f0cb2c7d4c2e6d036c0288e7f3eb18a3468259
Successfully built choldate
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 MB 35.0 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for knockpy: filename=knockpy-1.3.2-cp310-cp310-linux_x86_64.whl size=2455254 sha256=80cd3ad0

In [4]:
import shap
import numpy as np
import pandas as pd
import xgboost as xgb
import geopandas as gpd
from tqdm import tqdm
import knockpy.knockoffs as kf
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import matplotlib.colors as colors
from knockpy import KnockoffFilter
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from knockpy.knockoff_filter import KnockoffFilter
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold
from sklearn.multioutput import MultiOutputRegressor

import warnings
warnings.filterwarnings('ignore')

In [5]:
chunk_size = 5000
#df_chunks = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/Combined_df.csv", chunksize=chunk_size)
#df_chunks = pd.read_csv(r"\Users\gnane\Downloads\Copy of Combined_df.csv", chunksize=chunk_size)
df_chunks = pd.read_csv("/kaggle/input/mywork/Copy of Combined_df.csv", chunksize=chunk_size)
df = pd.concat(chunk for chunk in df_chunks)

In [6]:
medical_outcomes = ['o_depression_quantity_per_capita',
                    'o_anxiety_quantity_per_capita', 'o_opioids_quantity_per_capita',
                    'o_total_quantity_per_capita']#'o_diabetes_quantity_per_capita', 'o_hypertension_quantity_per_capita',
                    #'o_asthma_quantity_per_capita', 

In [7]:
np.random.seed(42)
additional_features_to_drop=['geography_code', 'lsoa21nm', 'geometry','income_category','year','centroid_x','centroid_y','o_ome_per_capita','o_diabetes_quantity_per_capita', 'o_hypertension_quantity_per_capita',
                    'o_asthma_quantity_per_capita']
X = df.drop(columns=medical_outcomes+additional_features_to_drop)
y = df[medical_outcomes]

In [8]:
def apply_knockoff_filter(X, y, fdr=0.2):
    scaler = StandardScaler()
    np.random.seed(42)
    predictors_scaled = scaler.fit_transform(X)
    all_selected_features = set()
    sampler = kf.GaussianSampler(predictors_scaled)
    X_knockoffs = sampler.sample_knockoffs()
    knockoff_filter = KnockoffFilter(fstat='lasso')
    features_by_outcome = {}

    for outcome in y.columns:
        selected_features = knockoff_filter.forward(
            X=predictors_scaled,
            Xk=X_knockoffs,
            y=y[outcome],
            fdr=fdr
        )
        selected_features_boolean = selected_features.astype(bool)
        selected_feature_names = X.columns[selected_features_boolean]
        features_by_outcome[outcome] = selected_feature_names.tolist()
        all_selected_features.update(selected_feature_names.tolist())

    unique_selected_features = list(all_selected_features)
    return features_by_outcome, X

In [9]:
def tune_xgboost(X, y, outcome, use_gpu=True):
    if use_gpu:
        try:
            gpu_check = xgb.XGBRegressor(tree_method='gpu_hist')
            print(f"GPU acceleration enabled for XGBoost - {outcome}")
        except Exception as e:
            print(f"Warning: GPU acceleration not available for {outcome}: {e}")
            use_gpu = False

    X_train, X_test, y_train, y_test = train_test_split(
        X, y[outcome], test_size=0.1, random_state=42
    )

    param_grid = {
        'n_estimators': [100, 200, 300, 400],
        'max_depth': [3, 5, 7, 9],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'subsample': [0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.7, 0.8, 0.9, 1.0]
    }

    xgb_model = xgb.XGBRegressor(
        objective='reg:squarederror',
        random_state=42,
        tree_method='gpu_hist' if use_gpu else 'hist',
        gpu_id=0 if use_gpu else None,
        predictor='gpu_predictor' if use_gpu else 'auto'
    )

    random_search = RandomizedSearchCV(
        xgb_model,
        param_distributions=param_grid,
        n_iter=10,
        cv=5,
        scoring='r2',
        random_state=42,
        n_jobs=-1
    )

    random_search.fit(X_train, y_train)
    best_params = random_search.best_params_
    return best_params, X_train, X_test, y_train, y_test

In [10]:
def create_rashomon_set(X_train, X_test, y_train, y_test, best_params, outcome, n_models=10, use_gpu=True):
    models = []
    performances = []
    for seed in range(n_models):
        model = xgb.XGBRegressor(
            **best_params,
            objective='reg:squarederror',
            random_state=seed,
            tree_method='gpu_hist' if use_gpu else 'hist',
            gpu_id=0 if use_gpu else None,
            predictor='gpu_predictor' if use_gpu else 'auto'
        )
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        models.append(model)
        performances.append(r2)
            # Rejection sampling: keep only top performing models
        performance_threshold = np.percentile(performances, 75)
        selected_models = [
            model for model, perf in zip(models, performances)
            if perf >= performance_threshold
        ]
    #print(f"R² scores for {outcome}: {performances}")
    return selected_models

In [11]:
def variable_importance_analysis(models, X_test, y_test, y_train, feature_names, outcome, top_n=10):
    feature_scores = {
        method: pd.DataFrame(index=feature_names)
        for method in ['permutation', 'shap', 'loco', 'cmr']
    }

    for i, model in enumerate(tqdm(models, desc=f"Processing Rashomon models for {outcome}")):
        print(f"\nEvaluating model {i+1}/{len(models)} for {outcome}")

        # 1. Permutation Importance
        perm = permutation_importance(
            model, X_test, y_test, scoring='r2', n_repeats=10,
            random_state=42, n_jobs=-1
        )
        feature_scores['permutation'][f'model_{i}'] = perm.importances_mean

        # 2. SHAP
        explainer = shap.Explainer(
            model,
            X_test.sample(n=min(200, len(X_test)), random_state=42),
            feature_perturbation="interventional"
        )
        shap_values = explainer(X_test)
        shap_mean = np.abs(shap_values.values).mean(axis=0)
        feature_scores['shap'][f'model_{i}'] = shap_mean

        # 3. LOCO
        loco_scores = []
        base_score = r2_score(y_test, model.predict(X_test))
        for col in feature_names:
            X_temp = X_test.drop(columns=col)
            # Create a new train/test split with the modified X
            X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(
                X_filtered.drop(columns=col), y[outcome], test_size=0.1, random_state=42
            )
            model_temp = xgb.XGBRegressor(**model.get_params())
            # Use the appropriate y_train for the modified X_train
            model_temp.fit(X_train_temp, y_train_temp)
            y_pred_temp = model_temp.predict(X_test_temp)
            loco_score = base_score - r2_score(y_test_temp, y_pred_temp)
            loco_scores.append(loco_score)
        feature_scores['loco'][f'model_{i}'] = loco_scores

        # 4. CMR Approximation
        cmr_scores = []
        for col in feature_names:
            X_imputed = X_test.copy()
            X_imputed[col] = np.random.permutation(X_imputed[col])
            cmr_score = base_score - r2_score(y_test, model.predict(X_imputed))
            cmr_scores.append(cmr_score)
        feature_scores['cmr'][f'model_{i}'] = cmr_scores

    # Combining all methods
    mean_ranks = pd.DataFrame(index=feature_names)
    for method, df in feature_scores.items():
        method_ranks = df.rank(ascending=False).mean(axis=1)
        mean_ranks[method] = method_ranks
    mean_ranks['mean_rank'] = mean_ranks.mean(axis=1)
    top_features = mean_ranks.sort_values('mean_rank').head(top_n)

    return top_features, mean_ranks, feature_scores

In [ ]:
# Process each outcome separately
results = {}
features_by_outcome, X_original = apply_knockoff_filter(X, y, fdr=0.2)

for outcome in y.columns:
    print(f"\nProcessing outcome: {outcome}")
    X_filtered = X_original[features_by_outcome[outcome]]

    # Tune XGBoost
    best_params, X_train, X_test, y_train, y_test = tune_xgboost(
        X_filtered, y, outcome, use_gpu=True
    )

    # Create Rashomon set
    rashomon_models = create_rashomon_set(
        X_train, X_test, y_train, y_test, best_params, outcome, use_gpu=True
    )

    # Variable importance analysis
    top_features, mean_ranks, feature_scores = variable_importance_analysis(
        models=rashomon_models,
        X_test=X_test,
        y_test=y_test,
        y_train=y_train,
        feature_names=X_filtered.columns.tolist(),
        outcome=outcome,
        top_n=10
    )

    results[outcome] = {
        'best_params': best_params,
        'models': rashomon_models,
        'features': features_by_outcome[outcome],
        'top_features': top_features,
        'mean_ranks': mean_ranks,
        'feature_scores': feature_scores
    }

    print(f"\nTop 10 Most Important Features for {outcome}:")
    print(top_features)

# Print overall summary
for outcome, result in results.items():
    print(f"\nSummary for {outcome}:")
    print(f"Best parameters: {result['best_params']}")
    print(f"Number of models: {len(result['models'])}")
    print(f"Selected features: {result['features']}")
    print(f"Top 10 features:\n{result['top_features']}")


Processing outcome: o_diabetes_quantity_per_capita
GPU acceleration enabled for XGBoost - o_diabetes_quantity_per_capita


Processing Rashomon models for o_diabetes_quantity_per_capita:   0%|          | 0/3 [00:00<?, ?it/s]


Evaluating model 1/3 for o_diabetes_quantity_per_capita


Processing Rashomon models for o_diabetes_quantity_per_capita:  33%|███▎      | 1/3 [17:54<35:49, 1074.92s/it]


Evaluating model 2/3 for o_diabetes_quantity_per_capita


...

Processing Rashomon models for o_asthma_quantity_per_capita: 100%|██████████| 3/3 [31:27<00:00, 629.18s/it]


Top 10 Most Important Features for o_asthma_quantity_per_capita:
                                   permutation       shap       loco  \
c_net_annual_income                   4.333333   7.333333   1.000000   
c_percent_commute_car                 1.666667   1.666667  12.666667   
c_percent_commute_bus                 5.000000   4.666667  16.666667   
e_leaf_area_index_high_vegetation     8.333333  13.000000   3.000000   
c_percent_tfw_2km_to_5km              9.666667  13.333333   3.333333   
c_percent_mixed                       3.000000   4.000000  31.666667   
e_lake_mix_layer_depth               10.000000  16.000000   8.333333   
e_no2                                 1.333333   1.333333  43.000000   
c_percent_10_years_or_more            8.000000   7.000000  27.666667   
c_percent_commute_metro_rail         13.000000  10.666667  15.666667   

                                         cmr  mean_rank  
c_net_annual_income                 4.666667   4.333333  
c_percent_commute_car    


Processing Rashomon models for o_depression_quantity_per_capita:   0%|          | 0/3 [00:00<?, ?it/s]


Evaluating model 1/3 for o_depression_quantity_per_capita


 23%|=====               | 1586/6751 [01:22<04:27]       

In [12]:
# Process each outcome separately
results = {}
features_by_outcome, X_original = apply_knockoff_filter(X, y, fdr=0.2)

for outcome in y.columns:
    print(f"\nProcessing outcome: {outcome}")
    X_filtered = X_original[features_by_outcome[outcome]]

    # Tune XGBoost
    best_params, X_train, X_test, y_train, y_test = tune_xgboost(
        X_filtered, y, outcome, use_gpu=True
    )

    # Create Rashomon set
    rashomon_models = create_rashomon_set(
        X_train, X_test, y_train, y_test, best_params, outcome, use_gpu=True
    )

    # Variable importance analysis
    top_features, mean_ranks, feature_scores = variable_importance_analysis(
        models=rashomon_models,
        X_test=X_test,
        y_test=y_test,
        y_train=y_train,
        feature_names=X_filtered.columns.tolist(),
        outcome=outcome,
        top_n=10
    )

    results[outcome] = {
        'best_params': best_params,
        'models': rashomon_models,
        'features': features_by_outcome[outcome],
        'top_features': top_features,
        'mean_ranks': mean_ranks,
        'feature_scores': feature_scores
    }

    print(f"\nTop 10 Most Important Features for {outcome}:")
    print(top_features)

# Print overall summary
for outcome, result in results.items():
    print(f"\nSummary for {outcome}:")
    print(f"Best parameters: {result['best_params']}")
    print(f"Number of models: {len(result['models'])}")
    print(f"Selected features: {result['features']}")


Processing outcome: o_depression_quantity_per_capita
GPU acceleration enabled for XGBoost - o_depression_quantity_per_capita


Processing Rashomon models for o_depression_quantity_per_capita:   0%|          | 0/3 [00:00<?, ?it/s]


Evaluating model 1/3 for o_depression_quantity_per_capita


Processing Rashomon models for o_depression_quantity_per_capita:  33%|███▎      | 1/3 [07:50<15:40, 470.05s/it]


Evaluating model 2/3 for o_depression_quantity_per_capita


...


Evaluating model 2/3 for o_total_quantity_per_capita


Processing Rashomon models for o_total_quantity_per_capita:  67%|██████▋   | 2/3 [21:57<10:57, 657.31s/it]


Evaluating model 3/3 for o_total_quantity_per_capita


Processing Rashomon models for o_total_quantity_per_capita: 100%|██████████| 3/3 [33:07<00:00, 662.34s/it]


Top 10 Most Important Features for o_total_quantity_per_capita:
                                           permutation       shap       loco  \
c_percent_commute_bicycle                     4.666667   3.000000   4.666667   
e_leaf_area_index_low_vegetation              4.000000   8.333333   3.333333   
c_net_annual_income                           6.000000   8.666667   2.000000   
c_percent_mixed                               1.333333   2.000000  23.666667   
e_surface_thermal_radiation_downwards_sum     1.666667   1.000000  30.333333   
e_surface_pressure                            3.333333  12.333333  26.333333   
e_evaporation_from_the_top_of_canopy_sum      7.666667   8.333333  32.666667   
c_percent_commute_bus                        13.333333  11.333333  23.333333   
c_percent_wfh                                14.000000   6.333333  28.666667   
e_surface_runoff_sum                         10.333333   8.000000  34.000000   

                                                 cmr  